In [113]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from random import randint
from time import sleep, strftime
import smtplib
from datetime import datetime

In [105]:
driver = webdriver.Chrome()
sleep(2)

In [106]:
kayak = "https://www.kayak.com/flights/GDL-MAD/2022-12-08/2023-01-26?sort=bestflight_a"
driver.get(kayak)

In [81]:
# Load more results to maximize the scraping

def load_more():
    #try:
    more_results = '//a[@class = "moreButton"]'
    results = driver.find_element(By.XPATH, more_results).click()
    print('sleeping...')
    #sleep(randint(1,5))
    #except

In [4]:
search_modes = {'best':'bestflight', 'cheap':'price', 'quick':'duration'}
def change_mode_with_button(mode):
        print('Changing to ' + mode + 'mode.')
        mode_button_xpath = f'//a[@data-code = "{search_modes[mode]}"]'
        driver.find_elements(By.XPATH, mode_button_xpath)[0].click()
        print('sleeping...')
        #sleep(randint(3,10))

In [89]:
all_field_xpath = {#'Results':'//*[@class = "resultWrapper"]',
                   'Depart_Time':'//*[@class = "section times"]//*[@class = "time-pair"][1]',  #'//*[@class = "depart-time base-time"]',
                   'Arrival_Time':'//*[@class = "section times"]//*[@class = "time-pair"][2]',  #'//*[@class = "arrival-time base-time"]',
                   'Stops':'//*[@class = "stops-text"]',
                   'Duration':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "top"]',
                   'Cities':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "bottom"]',
                   'Prices':'//*[@class="price option-text"]'}

In [90]:
def get_data(xpath):
    results_raw = driver.find_elements(By.XPATH, xpath)
    results_text = [" ".join(result.text.split('\n')) for result in results_raw if result.text != '']
    return results_text

In [91]:
def get_data_from_all_fields():
    departure_data = {}
    return_data = {}
    prices = []
    
    for key, val in all_field_xpath.items():
        results = get_data(val)
        
        # Price is the only field that's only one per result
        if key == 'Prices':
            prices = results
            continue
        
        departure_data[key] = results[::2]
        return_data[key] = results[1::2]
        
    return departure_data, return_data, prices

In [92]:
def display_dic(dic):
    for key, val in dic.items():
        print(f'{len(val)}, {key}: {val}')

In [93]:
def generate_flights_df(departure_data, return_data, prices):
    # Creating both data frames for departure and return and putting them together
    departure_df = pd.DataFrame.from_dict(departure_data)
    return_df = pd.DataFrame.from_dict(return_data)
    full_df = pd.concat([departure_df, return_df], axis=1)
    
    # Adding the multilavel column to the data frame
    column_array = [['Departure', 'Return'], departure_df.columns]
    column_index = pd.MultiIndex.from_product(column_array)
    full_df.columns = column_index
    
    # Adding the price column
    full_df['Price'] = prices
    
    return full_df

In [102]:
load_more()

sleeping...


In [108]:
departure_data, return_data, prices = get_data_from_all_fields()
display_dic(departure_data)
display_dic(return_data)

17, Depart_Time: ['2:00 pm', '12:25 pm', '2:13 pm', '12:25 pm', '12:25 pm', '2:00 pm', '12:25 pm', '12:25 pm', '2:00 pm', '2:00 pm', '12:25 pm', '2:00 pm', '12:25 pm', '8:30 pm', '12:25 pm', '2:00 pm', '8:30 pm']
17, Arrival_Time: ['8:00 am +1', '9:25 am +1', '4:05 pm +1', '9:25 am +1', '9:25 am +1', '8:00 am +1', '9:25 am +1', '9:25 am +1', '8:00 am +1', '8:00 am +1', '9:25 am +1', '8:00 am +1', '9:25 am +1', '5:45 pm +1', '9:25 am +1', '8:00 am +1', '5:45 pm +1']
17, Stops: ['nonstop', '1 stop', '2 stops', '1 stop', '1 stop', 'nonstop', '1 stop', '1 stop', 'nonstop', 'nonstop', '1 stop', 'nonstop', '1 stop', '1 stop', '1 stop', 'nonstop', '1 stop']
17, Duration: ['11h 00m', '14h 00m', '18h 52m', '14h 00m', '14h 00m', '11h 00m', '14h 00m', '14h 00m', '11h 00m', '11h 00m', '14h 00m', '11h 00m', '14h 00m', '14h 15m', '14h 00m', '11h 00m', '14h 15m']
17, Cities: ['GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MAD', 'GDL ‐ MA

In [109]:
flights_df = generate_flights_df(departure_data, return_data, prices)
flights_df

Departure                                                Return  \
   Depart_Time Arrival_Time    Stops Duration     Cities Depart_Time   
0      2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    10:55 am   
1     12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    12:40 pm   
2      2:13 pm   4:05 pm +1  2 stops  18h 52m  GDL ‐ MAD    10:45 am   
3     12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    11:55 pm   
4     12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD     1:05 pm   
5      2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    10:55 am   
6     12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    12:40 pm   
7     12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    12:40 pm   
8      2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    10:55 am   
9      2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    11:35 pm   
10    12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    11:40 am   
11     2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    11:35 pm   
12    12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    11:40 am   
13     8:30 pm   5:45 pm +1   1 stop  14h 15m  GDL ‐ MAD    12:40 pm   
14    12:25 pm   9:25 am +1   1 stop  14h 00m  GDL ‐ MAD    10:45 am   
15     2:00 pm   8:00 am +1  nonstop  11h 00m  GDL ‐ MAD    10:55 am   
16     8:30 pm   5:45 pm +1   1 stop  14h 15m  GDL ‐ MAD    11:55 pm   

                                               Price  
   Arrival_Time    Stops Duration     Cities          
0       7:27 pm   1 stop  15h 32m  MAD ‐ GDL  $1,202  
1      11:23 pm   1 stop  17h 43m  MAD ‐ GDL    $871  
2   12:50 pm +1  2 stops  33h 05m  MAD ‐ GDL    $798  
3    9:35 am +1   1 stop  16h 40m  MAD ‐ GDL    $868  
4      11:59 pm   1 stop  17h 54m  MAD ‐ GDL    $868  
5       7:27 pm   1 stop  15h 32m  MAD ‐ GDL  $1,202  
6      11:23 pm   1 stop  17h 43m  MAD ‐ GDL    $908  
7      11:23 pm   1 stop  17h 43m  MAD ‐ GDL    $921  
8       8:33 pm   1 stop  16h 38m  MAD ‐ GDL  $1,202  
9    8:30 am +1   1 stop  15h 55m  MAD ‐ GDL  $1,202  
10     11:23 pm  2 stops  18h 43m  MAD ‐ GDL    $921  
11   9:22 am +1   1 stop  16h 47m  MAD ‐ GDL  $1,202  
12     11:23 pm  2 stops  18h 43m  MAD ‐ GDL    $912  
13     11:23 pm   1 stop  17h 43m  MAD ‐ GDL  $1,032  
14     11:23 pm  2 stops  19h 38m  MAD ‐ GDL    $913  
15     10:30 pm   1 stop  18h 35m  MAD ‐ GDL  $1,202  
16   9:35 am +1   1 stop  16h 40m  MAD ‐ GDL  $1,008

In [143]:
class Flight_Scrapper:
    def __init__(self):
        # Creating a logger object to write progress
        self.logger = Logger()
        self.sleep_time_min = 5
        self.sleep_time_max = 11

        # Creating a web driver to browse the internet
        self.logger.log('Initiating Chrome driver instance')
        self.driver = webdriver.Chrome()
        self.current_url = ""

        # Parameters and info to do the flight search
        self.search_modes = {'best':'bestflight', 'cheap':'price', 'quick':'duration'}
        self.all_field_xpath = {'Wrappers':'//*[@class = "resultWrapper"]',
                                'Depart_Time':'//*[@class = "section times"]//*[@class = "time-pair"][1]',  #'//*[@class = "depart-time base-time"]',
                                'Arrival_Time':'//*[@class = "section times"]//*[@class = "time-pair"][2]',  #'//*[@class = "arrival-time base-time"]',
                                'Stops':'//*[@class = "section stops"]',
                                'Duration':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "top"]',
                                'Cities':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "bottom"]',
                                'Prices':'//*[@class="price option-text"]'}

        self.logger.log('Waiting to make sure driver starts properly, sleeping...')
        sleep(5)

    def load_more(self):
        self.logger.log('Attempting to load more results...')

        try:
            more_results_xpath = '//a[@class = "moreButton"]'
            self.driver.find_element(By.XPATH, more_results_xpath).click()

            self.logger.log('More results loaded, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except:
            self.logger.log('Something went wrong, could not get more results...')

    def change_mode_with_button(self,mode):
        self.logger.log('Changing to ' + mode + ' mode.')

        try:
            mode_button_xpath = f'//a[@data-code = "{self.search_modes[mode]}"]'
            self.driver.find_elements(By.XPATH, mode_button_xpath)[0].click()

            self.logger.log(f'{mode} mode active, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except Exception as e:
            print(e)
            self.logger.log('Something went wrong, could not change mode manually...')

    # Possible modes: cheap, best, quick
    def start_kayak(self, origin, destination, start_dates, end_dates, mode):
        # Check if user wants to check a mode in particular
        add_mode = ''
        if mode is not None:
            add_mode = '?sort={self.search_modes[mode]}_a'

        kayak = (f"https://www.kayak.com/flights/{origin}-{destination}/{start_dates}/{end_dates}{add_mode}")
        self.logger.log('Trying to get ' + kayak)

        try:
            # Getting the website
            self.driver.get(kayak)

            self.logger.log('Successfully accessed kayak, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except:
            self.logger.log(f'Something went wrong, could not get {kayak}...')

    def get_raw_data(self, xpath):
        results = self.driver.find_elements(By.XPATH, xpath)
        return [result.text for result in results]

    def get_data(self, xpath):
        results_raw = self.driver.find_elements(By.XPATH, xpath)
        results_text = [" ".join(result.text.split('\n')) for result in results_raw if result.text != '']
        return results_text

    def get_stops_from_wrappers(self, wrappers):
        departure_stops = ['' for _ in range(len(wrappers))]
        departure_n_stops = ['' for _ in range(len(wrappers))]
        return_stops = [0 for _ in range(len(wrappers))]
        return_n_stops = [0 for _ in range(len(wrappers))]

        base_index = 2

        for i, wrapper in enumerate(wrappers):
            departure_n_stops[i] = wrapper.split('\n')[base_index]
            if departure_n_stops[i] != 'nonstop': departure_stops[i] = wrapper.split('\n')[base_index+1]
            else: base_index -= 1

            return_n_stops[i] = wrapper.split('\n')[base_index+8]
            if return_n_stops[i] != 'nonstop': return_stops[i] = wrapper.split('\n')[base_index+9]
        
        return departure_n_stops, departure_stops, return_n_stops, return_stops

    def get_data_from_all_fields(self):
        departure_data = {}
        return_data = {}
        prices = []
        wrappers = []
        
        self.logger.log(f'Getting info from fields: {list(self.all_field_xpath)}')
        for key, val in self.all_field_xpath.items():
            # We dont actually want to process the wrapper so will call the raw method
            if key == 'Wrappers':
                wrappers = self.driver.find_elements(By.XPATH, self.all_field_xpath['Wrappers'])
                wrappers = [x.text for x in wrappers]
                continue

            results = self.get_data(val)
            
            # Price is the only field that's only one per result
            if key == 'Prices':
                prices = results
                continue
            
            departure_data[key] = results[::2]
            return_data[key] = results[1::2]
            
        return wrappers, departure_data, return_data, prices

    def _display_dic(self, dic):
        for key, val in dic.items():
            print(f'{len(val)}, {key}')

    def generate_flights_df(self, wrappers, departure_data, return_data, prices):
        self.logger.log('Creating data frames from fetched info')

        print(len(wrappers), len(prices))
        # Display dictionary lenghts
        print('departure')
        self._display_dic(departure_data)
        print('return')
        self._display_dic(return_data)

        # Creating both data frames for departure and return and putting them together
        try:
            departure_df = pd.DataFrame.from_dict(departure_data)
            return_df = pd.DataFrame.from_dict(return_data)
            full_df = pd.concat([departure_df, return_df], axis=1)
            self.logger.log('Successfully created dataframe...')
        except:
            self.logger.log('Dictionary dimensions are incorrect, trying to fix from raw wrapper data...')

            try:
                departure_n_stops, _, return_n_stops, _ = self.get_stops_from_wrappers(wrappers)
                departure_data['Stops'] = departure_n_stops
                return_data['Stops'] = return_n_stops

                departure_df = pd.DataFrame.from_dict(departure_data)
                return_df = pd.DataFrame.from_dict(return_data)
                full_df = pd.concat([departure_df, return_df], axis=1)
                self.logger.log('Successfully created dataframe...')
            except Exception as e:
                print(e)
                print('departure')
                self._display_dic(departure_data)
                print('return')
                self._display_dic(return_data)
        
        # Adding the multilavel column to the data frame
        column_array = [['Departure', 'Return'], departure_df.columns]
        column_index = pd.MultiIndex.from_product(column_array)
        full_df.columns = column_index
        
        # Adding the price column
        full_df['Price'] = prices
        
        return full_df
    
    def get_flights(self, origin, destination, departure_date, return_date, load_attemps=1, modes=['best', 'cheap', 'quick']):
        self.logger.log(f'STARTING getting flights from {origin} to {destination}, from {departure_date} to {return_date}')

        dfs = []
        full_wrappers = []

        # Getting results for passed modes
        for mode in modes:
            self.logger.log(f'TRYING info scrape for mode {mode}')
            # Getting the webpage
            self.start_kayak(origin, destination, departure_date, return_date, None)

            # Clicking the current mode button
            self.change_mode_with_button(mode)

            # Loading more results +15 results per load approx.
            for _ in range(load_attemps):
                self.load_more() 

            wrappers, departure_data, return_data, prices = self.get_data_from_all_fields()
            dfs.append(self.generate_flights_df(wrappers, departure_data, return_data, prices))
            full_wrappers += wrappers

        # Concatenating all dataframes into one
        full_df = pd.concat(dfs, ignore_index=True)
        return full_df, wrappers

In [144]:
class Logger:
    def __init__(self, path='.'):
        # Creating the log file where everything will be documented
        time = datetime.now()
        file_name = f"{path}/FlightFinder_" + time.strftime("%d-%m-%Y_%H-%M-%S") + ".log"
        self.log_file = open(file_name, 'w')

        # Writing important headers to the log file
        self.log_file.write(f'Execution started on {time.strftime("%d/%m/%Y at %H:%M:%S")}\n\n')

    def log(self, data):
        time = datetime.now()
        self.log_file.write(f'{time.strftime("%H:%M:%S")} > ')
        self.log_file.write(data + '\n')
        print(data)

In [146]:
scrapper = Flight_Scrapper()

flights, wrappers = scrapper.get_flights('GDL', 'MAD', '2022-12-08', '2023-01-26', load_attemps=3)

Initiating Chrome driver instance
Waiting to make sure driver starts properly, sleeping...
STARTING getting flights from GDL to MAD, from 2022-12-08 to 2023-01-26
TRYING info scrape for mode best
Trying to get https://www.kayak.com/flights/GDL-MAD/2022-12-08/2023-01-26
Successfully accessed kayak, sleeping...
Changing to best mode.
best mode active, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Getting info from fields: ['Wrappers', 'Depart_Time', 'Arrival_Time', 'Stops', 'Duration', 'Cities', 'Prices']
Creating data frames from fetched info
62 62
departure
62, Depart_Time
62, Arrival_Time
62, Stops
62, Duration
62, Cities
return
62, Depart_Time
62, Arrival_Time
62, Stops
62, Duration
62, Cities
Successfully created dataframe...
TRYING info scrape for mode cheap
Trying to get https://www.kayak.com/flights/GDL-MAD/2022-

In [147]:
flights.to_excel('My_output.xlsx', index='False')
flights.iloc[1]

Departure  Depart_Time       12:25 pm
           Arrival_Time    9:25 am +1
           Stops           1 stop DFW
           Duration           14h 00m
           Cities           GDL ‐ MAD
Return     Depart_Time       11:55 pm
           Arrival_Time    9:35 am +1
           Stops           1 stop MEX
           Duration           16h 40m
           Cities           MAD ‐ GDL
Price                            $866
Name: 1, dtype: object

In [149]:
def sort_by_price(flights):
    def extract_price(s):
            return int(s[1:].replace(',', ''))
        
    sorted_flights = flights.copy()

    sorted_flights['real_price'] = sorted_flights['Price'].apply(extract_price)
    sorted_flights.sort_values('real_price', inplace=True)
    sorted_flights.drop('real_price', axis=1, inplace=True)
    
    return sorted_flights

In [150]:
sorted_flights = sort_by_price(flights)

In [151]:
sorted_flights

Departure                                                         Return  \
   Depart_Time Arrival_Time             Stops Duration     Cities Depart_Time   
2      2:13 pm   4:05 pm +1  2 stops ATL, AMS  18h 52m  GDL ‐ MAD    10:45 am   
21    12:25 pm   9:25 am +1        1 stop DFW  14h 00m  GDL ‐ MAD     4:40 pm   
66     6:02 am   9:25 am +1        1 stop DFW  20h 23m  GDL ‐ MAD     1:05 pm   
42    12:25 pm   9:25 am +1        1 stop DFW  14h 00m  GDL ‐ MAD     4:40 pm   
34    12:25 pm   9:25 am +1        1 stop DFW  14h 00m  GDL ‐ MAD    11:55 pm   
..         ...          ...               ...      ...        ...         ...   
7      2:00 pm   8:00 am +1           nonstop  11h 00m  GDL ‐ MAD    10:55 am   
4      2:00 pm   8:00 am +1           nonstop  11h 00m  GDL ‐ MAD    10:55 am   
37     2:00 pm   8:00 am +1           nonstop  11h 00m  GDL ‐ MAD    11:35 pm   
45     6:28 pm   4:20 pm +1        1 stop MEX  14h 52m  GDL ‐ MAD    10:55 am   
31     2:00 pm   8:00 am +1           nonstop  11h 00m  GDL ‐ MAD    11:45 pm   

                                                        Price  
   Arrival_Time             Stops Duration     Cities          
2   12:50 pm +1  2 stops JFK, ATL  33h 05m  MAD ‐ GDL    $798  
21   7:20 am +1        1 stop MEX  21h 40m  MAD ‐ GDL    $866  
66     11:59 pm        1 stop MEX  17h 54m  MAD ‐ GDL    $866  
42   9:35 am +1        1 stop MEX  23h 55m  MAD ‐ GDL    $866  
34   2:50 pm +1        1 stop MEX  21h 55m  MAD ‐ GDL    $866  
..          ...               ...      ...        ...     ...  
7       8:33 pm        1 stop MEX  16h 38m  MAD ‐ GDL  $1,202  
4       7:27 pm        1 stop MEX  15h 32m  MAD ‐ GDL  $1,202  
37  12:22 pm +1        1 stop MEX  19h 47m  MAD ‐ GDL  $1,202  
45      7:27 pm        1 stop MEX  15h 32m  MAD ‐ GDL  $1,286  
31   8:30 am +1        1 stop MTY  15h 45m  MAD ‐ GDL  $1,359  

[92 rows x 11 columns]